# Analysis of Sampling Rate

Aim: Analyze Sampling Rates by ICUSTAY ID an Item Id to generate an overview

## Load and Prepare Data

In [ ]:
import pandas as pd
import pyarrow as pa

# Read chartevents_subset from parquet file to pandas data frame
chartevents_subset = pd.read_parquet('./data/chartevents_subset.parquet', engine='pyarrow')
unique_icu_stays = pd.read_parquet('./data/unique_icustays_in_chartevents_subset.parquet', engine='pyarrow')

In [ ]:

#select ICU_stay ids for analysis
icustayid_filter = unique_icu_stays.ICUSTAY_ID

# Filter by ICU_stay
sampling_rate_data = chartevents_subset[chartevents_subset.ICUSTAY_ID.isin(icustayid_filter)].copy()

In [ ]:
# Sampling Rate Analysis is only being conducted on the values, not thresholds
# Filter for item ids that refer to value
itemids_for_values_filter = [220045, 220179, 220277]

#Create empty data frame that will hold the sampling rate for each selected icu stay and item id

sampling_rates_for_icustay_itemid = pd.DataFrame(columns=["ICUSTAY_ID", "ITEMID","FIRST_TSP","LAST_TSP","ICUSTAY_DURATION(h)","N_MEASUREMENTS","SAMPLING_RATE"])

## Generate Data Frame with Sampling Rate

In [ ]:
for icustay in icustayid_filter:
    # filter only on values
    for itemid in itemids_for_values_filter:
        first_timestamp = sampling_rate_data[(sampling_rate_data['ICUSTAY_ID'] == icustay) & (sampling_rate_data['ITEMID'] == itemid)].CHARTTIME.min()
        last_timestamp = sampling_rate_data[(sampling_rate_data['ICUSTAY_ID'] == icustay) & (sampling_rate_data['ITEMID'] == itemid)].CHARTTIME.max()
        duration = last_timestamp-first_timestamp
        duration_in_s = duration.total_seconds()
        duration_in_h = divmod(duration_in_s, 3600)[0]

        #sampling rate - avg. # of samples obtained in 1 hours
        n_measurements = len(sampling_rate_data[(sampling_rate_data['ICUSTAY_ID'] == icustay) & (sampling_rate_data['ITEMID'] == itemid)])

        # Create Logic for cases in which we only have one measurement and first timestamp = last timestamp
        # we would divide by 0 as we have 1 measurement within 0 hours
        # Solution for now: write n_measurements to sampling rate
        if (duration_in_h == 0):
            sampling_rate = n_measurements
        else:
            sampling_rate = n_measurements/duration_in_h

        
        #append values as row to data frame
        a_new_row= {"ICUSTAY_ID":icustay,"ITEMID":itemid,"FIRST_TSP":first_timestamp,"LAST_TSP":last_timestamp,"ICUSTAY_DURATION(h)":duration_in_h,"N_MEASUREMENTS":n_measurements,  "SAMPLING_RATE":sampling_rate}
        a_new_row_series = pd.Series(a_new_row, name=icustay)
        sampling_rates_for_icustay_itemid = sampling_rates_for_icustay_itemid.append(a_new_row_series)

In [ ]:
sampling_rates_for_icustay_itemid.groupby('ITEMID')['ICUSTAY_DURATION(h)'].describe()

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa

# Save chartevents_subset as parquet file
pd.DataFrame(sampling_rates_for_icustay_itemid).to_parquet('./data/sampling_rates_for_icustay_itemid.parquet', engine='pyarrow')

In [ ]:
# create subsets for item Ids
sampling_rates_for_icustay_HR = sampling_rates_for_icustay_itemid[sampling_rates_for_icustay_itemid['ITEMID'] == 220045]
sampling_rates_for_icustay_O2 = sampling_rates_for_icustay_itemid[sampling_rates_for_icustay_itemid['ITEMID'] == 220277]
sampling_rates_for_icustay_NBP = sampling_rates_for_icustay_itemid[sampling_rates_for_icustay_itemid['ITEMID'] == 220179]

## Plot the sampling rate by Item Id

In [ ]:
# Imports
import seaborn as sns
import matplotlib.pyplot as plt

# Set variables
title = "Sampling Rate by Item Id"
xlabel = "Item Id"
ylabel = "AVG # of samples obtained in 1 hour"
plotdata = sampling_rates_for_icustay_itemid
xvalue = "ITEMID"
yvalue = "SAMPLING_RATE"

# Config figure
sns.set_style("whitegrid")
fig, ax = plt.subplots(
    figsize = (10, 5), 
    dpi = 72 # e.g. 72 for screen, 300 for print
    )
ax = sns.boxplot( # Insert on of: sns.stripplot , sns.boxplot , sns.violinplot
    data = plotdata, 
    x = xvalue,
    y = yvalue, # Comment out if no stratification is to be performed based on yvalue
    palette = sns.color_palette("colorblind")
    )
ax.set_title(title, fontweight='bold', color= 'black', fontsize=14, y=1.05)
ax.set_xlabel(xlabel, fontsize=12, labelpad=15)
ax.set_ylabel(ylabel, fontsize=12, labelpad=15) # Comment out if no stratification is to be performed based on yvalue
ax.grid(b=True, which='both')
ax.margins(.1)

# Plot figure
plt.show(fig)

## Validate Data

In [ ]:
# check icus stays where we see nan values

sampling_rate_nan = sampling_rates_for_icustay_itemid[sampling_rates_for_icustay_itemid['FIRST_TSP'].isnull()]
sampling_rate_nan['ICUSTAY_ID'].value_counts()
# we have 306 unique ICU Stays for which at least one parameter is missing



In [ ]:
# Check for ICU STay: 204787 and ITEM ID:220179
# Sampling Data Frame shows 
selected_icustay = sampling_rate_data[(sampling_rate_data['ICUSTAY_ID']==204787) & (sampling_rate_data['ITEMID']==220179)]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Set variables
title = "History of Non Invasive Blood Pressure systolic of ICU stay 253126"
xlabel = "Time"
ylabel = "mmHg"
plotdata = selected_icustay
xvalue = "CHARTTIME"
yvalue = "VALUENUM"
#huevalue = "ITEMID_LABEL"

# Config figure
sns.set_style("whitegrid")
fig, ax = plt.subplots(
    figsize = (11, 5), 
    dpi = 72 # e.g. 72 for screen, 300 for print
    )
ax = sns.lineplot(
    data = plotdata, 
    x = xvalue,
    y = yvalue, 
    #hue = huevalue,
    #style= huevalue,
    drawstyle = 'steps-post', # Interpolate missing values by using the last available value
    markers = ['p','^','v'],
    markersize = 5,
    dashes = False,
    #palette = [sns.color_palette("colorblind")[0],sns.color_palette("colorblind")[1],sns.color_palette("colorblind")[2]]
    )

plt.legend(title = None, bbox_to_anchor=(1.02, 0.3), loc='upper left', borderaxespad=0)
ax.set_title(title, fontweight='bold', color= 'black', fontsize=14, y=1.05)
ax.set_xlabel(xlabel, fontsize=12, labelpad=15)
ax.set_ylabel(ylabel, fontsize=12, labelpad=15)
plt.xticks(rotation = 90)

# Plot figure
plt.show(fig)